### Config

In [1]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from data_processing import chunk_text
from groq import Groq
from langchain_groq import ChatGroq
from langchain.schema import Document
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.tavily_search import TavilySearchResults
from pprint import pprint
from fastembed import TextEmbedding
import os

/Users/linhbk/Documents/python projects/NCKH_2024/RagAgent/agent_libs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Embedding & call llm

In [2]:
embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Fetching 5 files: 100%|██████████| 5/5 [00:06<00:00,  1.35s/it]


In [3]:
import os
groq_api_key = os.environ['GROQ_API_KEY']
llm = ChatGroq(model_name='Llama3-8b-8192', api_key=groq_api_key)

###  Chunking text

In [4]:
# raw_texts = process_and_chunk_data('data')
raw_texts = chunking_txt_file('data/luat_dat_dai.txt')
doc_splits = [Document(page_content=chunk) for chunk in raw_texts]
doc_splits[-1]

Document(metadata={}, page_content='toàn bộ hoặc một phần dự án bất động sản được kế thừa quyền và nghĩa vụ về đất đai của bên chuyển nhượng dự án. Luật này được Quốc hội nước Cộng hòa xã hội chủ nghĩa Việt Nam khóa XV, kỳ họp bất thường lần thứ năm thông qua ngày 18 tháng 01 năm 2024.')

In [5]:
doc_count = len(doc_splits)
doc_count

1295

## RAG

### save vector db in persist files

In [6]:
persist_directory = './real_estate_db/luat_dat_dai'

### chromadb

In [7]:
# # create db
# vectorstore_created = Chroma.from_documents(documents=doc_splits,
#                                     embedding=embed_model,
#                                     persist_directory=persist_directory,
#                                     collection_name="local-rag")
# vectorstore_created.persist()

In [8]:
# call from existed db
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embed_model, collection_name="local-rag")
# vectorstore.get()

/var/folders/cc/64gffy7j1_79tlsdxkhtcjg00000gp/T/ipykernel_1576/87617570.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embed_model, collection_name="local-rag")


In [9]:
print("Number of stored documents:", vectorstore._collection.count())
print("First document:", doc_splits[-1].page_content if doc_splits else "No documents found!")
print("Files in persistence directory:", os.listdir(persist_directory))

Number of stored documents: 1295
First document: toàn bộ hoặc một phần dự án bất động sản được kế thừa quyền và nghĩa vụ về đất đai của bên chuyển nhượng dự án. Luật này được Quốc hội nước Cộng hòa xã hội chủ nghĩa Việt Nam khóa XV, kỳ họp bất thường lần thứ năm thông qua ngày 18 tháng 01 năm 2024.
Files in persistence directory: ['chroma.sqlite3', 'b3ca9191-a5a0-427b-b596-7f502b6bcc0c']


In [10]:
from chromadb import PersistentClient
client = PersistentClient(path=persist_directory)
collections = client.list_collections()
print("Available collections:", collections)


Available collections: ['local-rag']


In [11]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever_lambda = RunnableLambda(lambda x: retriever.get_relevant_documents(x["question"]))

In [12]:
# from langchain.schema import Document
# from langchain_core.runnables import RunnableLambda

# # convert chunks to document
# def format_text_chunks(text_chunks):
#     return [Document(page_content=chunk) for chunk in text_chunks]

# retriever_lambda = RunnableLambda(lambda x: format_text_chunks(retriever.get_relevant_documents(x["question"])))

### Router

In [13]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore for questions on real estate laws in Vietnam. You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
    no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)
start = time.time()
question_router = router_prompt | llm | JsonOutputParser()

# test
question = "“Người sử dụng đất” được hiểu như thế nào theo quy định của Luật Đất đai năm 2024?"
print(question_router.invoke({"question": question}))
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

{'datasource': 'vectorstore'}
The time required to generate response by Router Chain in seconds:0.40052318572998047


### Generator

In [14]:
from langchain_core.runnables import RunnableLambda

qa_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just refuse answer in polite and friendly. 
    Answer question in detailed, make sure references vietnamese's law that prove for your answer. 
    For example:
    Question: Phạm vi điều chỉnh và đối tượng áp dụng Luật Đất đai năm 2024 là gì?
    Answer: '
    Điều 1. Phạm vi điều chỉnh
    Luật này quy định về chế độ sở hữu đất đai, quyền hạn và trách nhiệm của Nhà nước đại diện chủ sở hữu toàn dân về đất đai và thống nhất quản lý về đất đai, chế độ quản lý và sử dụng đất đai, quyền và nghĩa vụ của công dân, người sử dụng đất đối với đất đai thuộc lãnh thổ của nước Cộng hòa xã hội chủ nghĩa Việt Nam.
    Điều 2. Đối tượng áp dụng
    1. Cơ quan nhà nước thực hiện quyền hạn và trách nhiệm đại diện chủ sở hữu toàn dân về đất đai, thực hiện nhiệm vụ thống nhất quản lý nhà nước về đất đai.
    2. Người sử dụng đất.
    3. Các đối tượng khác có liên quan đến việc quản lý, sử dụng đất đai.'
    Answer in professional in vietnamese.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Chain
start = time.time()
rag_chain = (
    {"question": lambda x: x["question"], "context": retriever_lambda}
    | qa_prompt
    | llm
    | StrOutputParser()
)

# test
# question = "luật nhà ở 2024"
response = rag_chain.invoke({"question": question})
print(response)

/var/folders/cc/64gffy7j1_79tlsdxkhtcjg00000gp/T/ipykernel_1576/2309398310.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_lambda = RunnableLambda(lambda x: retriever.get_relevant_documents(x["question"]))


Theo quy định của Luật Đất đai năm 2024, "Người sử dụng đất" được hiểu là tổ chức, cá nhân được Nhà nước giao đất, cho thuê đất hoặc nhận chuyển nhượng quyền sử dụng đất, có quyền và nghĩa vụ được quy định tại Luật này.

Tại Điều 3 của Luật Đất đai năm 2024, có quy định:

"1. Người sử dụng đất là tổ chức, cá nhân được Nhà nước giao đất, cho thuê đất, nhận chuyển nhượng quyền sử dụng đất hoặc được Nhà nước công nhận là người sử dụng đất.

2. Người sử dụng đất có quyền và nghĩa vụ được quy định tại Luật này, trừ trường hợp quy định tại khoản 3 Điều 33 của Luật này."

Tại khoản 3 Điều 33 của Luật Đất đai năm 2024, có quy định:

"3. Người sử dụng đất không có tư cách là người sử dụng đất trong trường hợp sau:

* Đất được Nhà nước thu hồi để thực hiện dự án quốc phòng, an ninh, công công, phát triển kinh tế, cải thiện môi trường, bảo đảm an toàn và trật tự;

* Đất được Nhà nước thu hồi để phát triển kinh tế, cải thiện môi trường, bảo đảm an toàn và trật tự;

* Đất được Nhà nước thu hồi để t

### Retriever

In [15]:
retrieval_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. \n
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)
start = time.time()
retrieval_grader = retrieval_grader_prompt | llm | JsonOutputParser()

# test
# question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
end = time.time()
print(f"The time required to generate response by the retrieval grader in seconds:{end - start}")


{'score': 'no'}
The time required to generate response by the retrieval grader in seconds:0.3015322685241699


In [16]:
docs[0]

Document(metadata={}, page_content='vốn góp là giá trị quyền sử dụng đất theo quy định của pháp luật. 6. Thực hiện các quyền của người sử dụng đất theo quy định của Luật này. Điều 24. Quyền tiếp cận thông tin đất đai 1. Công dân được tiếp cận các thông tin đất đai sau đây: a) Quy hoạch, kế hoạch sử dụng đất, các quy hoạch có liên quan đến sử dụng đất đã được cơ quan nhà nước có thẩm quyền quyết định, phê duyệt; b) Kết quả thống kê, kiểm kê đất đai; c) Giao đất, cho thuê đất; d) Bảng giá đất đã được công bố; đ) Phương án bồi')

### Hallucination

In [17]:
hallucination_grader_prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)
start = time.time()
hallucination_grader = hallucination_grader_prompt | llm | JsonOutputParser()

# test
hallucination_grader_response = hallucination_grader.invoke({"documents": docs, "generation": response})
end = time.time()
print(f"The time required to generate response by the generation chain in seconds:{end - start}")
print(hallucination_grader_response)

The time required to generate response by the generation chain in seconds:0.3920912742614746
{'score': 'yes'}


### ANswer grader

In [18]:
answer_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = answer_grader_prompt | llm | JsonOutputParser()

# test
answer_grader_response = answer_grader.invoke({"question": question,"generation": response})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

The time required to generate response by the answer grader in seconds:0.3267068862915039
{'score': 'yes'}


### Websearch tool

In [19]:
tavily_api_key = os.environ['TAVILY_API_KEY']
web_search_tool = TavilySearchResults(k=3)


Single test web_search_tool

In [20]:
import os
from langchain.tools import TavilySearchResults

def search_web(query: str, k: int = 3):
    """
    Searches the web using Tavily API for the given query.
    
    Parameters:
    - query (str): The search query.
    - k (int): Number of search results to return (default is 3).
    
    Returns:
    - list: A list of search result snippets.
    """
    tavily_api_key = os.environ['TAVILY_API_KEY']
    web_search_tool = TavilySearchResults(k=k)
    
    try:
        results = web_search_tool.run(query)
        return results
    except Exception as e:
        print(f"Error while searching: {e}")
        return []

query = "TÌnh hình bất đọngo sản 2024"
search_results = search_web(query)
print(search_results)


[{'url': 'https://kinhtedothi.vn/nhin-lai-thi-truong-bat-dong-san-nam-2024.html', 'content': 'Tính chung cả năm 2024, toàn thị trường ghi nhận khoảng gần 81.000 sản phẩm chào bán, tăng hơn 40% so với năm 2023. Trong đó, có 65,376 sản phẩ'}, {'url': 'https://laodong.vn/bat-dong-san/toan-canh-thi-truong-bat-dong-san-9-thang-dau-nam-2024-1397317.ldo', 'content': 'Theo đó, mức độ quan tâm đất trong quý III/2024 dự kiến tăng 49% so với cùng kỳ 2023, nhà riêng tăng 25%, chung cư tăng 24%, biệt thự tăng 22%.'}, {'url': 'https://baochinhphu.vn/chuyen-dong-tich-cuc-cua-thi-truong-bat-dong-san-trong-quy-iii-2024-102241015110847106.htm', 'content': 'Đại diện Viện nghiên cứu Đánh giá thị trường bất động sản Việt Nam cho biết, quý 3 năm 2024, thị trường BĐS nhà ở tiếp tục ghi nhận nguồn cung đạt mức 22.412 sản phẩm được chào bán trên thị trường, với khoảng 14.750 sản phẩm mở bán mới, giảm 25% so với quý trước, nhưng đã tăng 60% so với cùng kỳ năm 2023. Nghiên cứu về chỉ số giá căn hộ, phản ánh mức 

## LangGraphh

In [21]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]
    iterations: int

### Nodes

In [22]:
from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}
#
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    iterations = state.get("iterations", 0) + 1
    
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation, "iterations": iterations}
#
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    
    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}
#
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])
    iterations = state.get("iterations", 0) + 1

    # Web search
    docs = web_search_tool.run({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question, "iterations": iterations}

### Condition edges

In [23]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})  
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


In [24]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


In [25]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    iterations = state.get("iterations", 0)
    
    if iterations >= 10:
        print("GETTING MAX ATTEMPTS")
        return "end"

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

### Add node

In [26]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

### Entry & End points

In [27]:
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
        "end": END
    },
)

In [28]:
app = workflow.compile()

## Test

In [29]:
from pprint import pprint

inputs = {"question": "người dân có quyền sử dụng đất bao nhiêu năm?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
    # if "generation" in value:
    #     pprint(value["generation"])
    # else:
    #     pprint("Hiện mình chưa có thông tin về câu hỏi của bạn. Bạn có thể thử lại với câu hỏi khác không?")
pprint(value["generation"])

---ROUTE QUESTION---
người dân có quyền sử dụng đất bao nhiêu năm?
{'datasource': 'vectorstore'}
vectorstore
---ROUTE QUESTION TO RAG---
---RETRIEVE---
'Finished running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
'Finished running: grade_documents:'
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
'---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---'
'Finished running: generate:'
---GENERATE---
---CHECK HALLUCINATIONS---
'---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---'
'Finished running: generate:'
---GENERATE---
---CHECK HALLUCINATIONS---
'---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---'
'Finished running: generate:'
---GENERATE---
---CHECK HALLUCINATIONS---
'---DECISION: GENERA